# Jupyter Notebook for Counting Building Occupancy from Polaris Traffic Simulation Data

This notebook will load a Polaris SQLlite data file into a Pandas data frame using sqlite3 libraries and count the average number of people in each building in each hour of the simulation.

For help with Jupyter notebooks

For help on using sql with Pandas see
http://www.pererikstrandberg.se/blog/index.cgi?page=PythonDataAnalysisWithSqliteAndPandas

For help  on data analysis with Pandas see
http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/Index.ipynb

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Create your connection.  Assumes data is in a parallel subdirectory to this one
cnx = sqlite3.connect('..\data\detroit-Demand.sqlite')

In [3]:
# exract all the beginning locations of the building simulation
beginning_location = pd.read_sql_query("SELECT * FROM Beginning_Location_All", cnx)

DatabaseError: Execution failed on sql 'SELECT * FROM Beginning_Location_All': no such table: Beginning_Location_All

In [4]:
trips = pd.read_sql_query("SELECT start, end, origin, destination, person FROM Trip", cnx)
trips["start_hr"] = trips.start // 3600
trips["end_hr"] = trips.end // 3600

In [5]:
# create the data frames that have the counts of things grouped by start hr & origin and end hr & destination
departs = trips.groupby(['start_hr','origin']).size().reset_index(name='countleave')
arrives = trips.groupby(['end_hr','destination']).size().reset_index(name='countarrive')

In [6]:
departm = {}
arrivem = {}
for i in range(24):
    departm[i] = pd.merge(
        beginning_location, departs[departs.start_hr == i],
        left_on='location', right_on='origin', how='left'
    ).fillna(0)
    arrivem[i] = pd.merge(
        beginning_location, arrives[arrives.end_hr == i],
        left_on='location', right_on='destination', how='left'
    ).fillna(0)

NameError: name 'beginning_location' is not defined

In [8]:
occm = {}
occm[0] = departm[0].occupants + arrivem[0].countarrive - departm[0].countleave
for i in range(1, 24):
    occm[i] = occm[i-1] + arrivem[i].countarrive - departm[i].countleave

In [9]:
occupancy = pd.DataFrame(occm)

In [10]:
occupancy["location"]=beginning_location.location
occupancy['land_use']=beginning_location.land_use

In [11]:
cols=['location', 'land_use',0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
occupancy = occupancy[cols]

In [12]:
# Remove locations with no occupancy activity
occupancy_clean = occupancy[occupancy.iloc[:, 2:].abs().sum(axis=1) != 0]

In [13]:
# Group locations by land use type
land_uses = occupancy_clean.land_use.unique()
occu_profiles = {}
for land in land_uses:
    profiles = occupancy_clean[occupancy_clean.land_use == land].drop('land_use', axis=1)
    occu_profiles[land] = profiles.set_index('location')

In [14]:
hr = range(24)

In [15]:
# Occupancy schedules by land use type
fig, axs = plt.subplots(5, 2, figsize=(16, 20))
axs_flat = [x for y in axs for x in y]
for i, ax in enumerate(axs_flat[:-1]):
    land_type = list(occu_profiles.keys())[i]
    df = occu_profiles[land_type]
    ax.step(hr, df.mean(axis=0), color=[0, 0.45, 0.7], label='average')
    ax.fill_between(
        hr, df.min(axis=0), df.max(axis=0),
        step='pre', color=[0, 0.45, 0.7], alpha=0.5, label='range'
    )
    ax.set_xticks(range(0, 24, 2))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('hour', fontsize=14)
    ax.set_ylabel('occupancy', fontsize=14)
    ax.legend(fontsize=14)
    ax.set_xlim([0,23])
    ax.set_ylim([df.min().min() - 1,df.max().max() + 1])
    ax.set_title(land_type, fontsize=18)
fig.tight_layout()
fig.savefig('.\output\occupancy_by_land_use_type.png', bbox_inches='tight', dpi=600)
plt.close()

In [16]:
# Histogram of maximum occupancy by land use type
fig, axs = plt.subplots(5, 2, figsize=(12, 20))
axs_flat = [x for y in axs for x in y]
for i, ax in enumerate(axs_flat[:-1]):
    land_type = list(occu_profiles.keys())[i]
    df = occu_profiles[land_type]
    ax.hist(
        x=df.max(axis=1).values, color=[0, 0.45, 0.7],
        alpha=0.5, label='all buildings'
    )
    ax.axvline(
        x=df.max(axis=1).mean(), color=[0, 0.45, 0.7],
        linewidth=2, label='average'
    )
    # ax.set_xticks(range(0, 24, 1))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('Maximum occpuancy', fontsize=14)
    ax.set_ylabel('Counts', fontsize=14)
    ax.legend(fontsize=14)
    ax.set_title(land_type, fontsize=18)
fig.tight_layout()
fig.savefig('.\output\maximum_occupancy_by_land_use_type.png', bbox_inches='tight', dpi=600)
plt.close()

In [17]:
# Export occupancy profile data
for key, value in occu_profiles.items():
    csv_out = '.\output\Detroit_occupancy_{}.csv'.format(key.lower())
    value.to_csv(csv_out)

Clean and normalization

In [18]:
# Extract schedules with 4+ maximum occupancy and 0+ minimum occupancy
occu_profiles_clean = {}
for key, value in occu_profiles.items():
    occu_profiles_clean[key] = value[(value.max(axis=1) >= 4) & (value.min(axis=1) >= 0)]

In [19]:
# Normalize by maximum occupancy
occu_profiles_clean_norm = {}
for key, value in occu_profiles_clean.items():
    occu_profiles_clean_norm[key] = value.div(value.max(axis=1), axis=0)

In [20]:
land_types = ['BUSINESS', 'RESIDENTIAL-MULTI']

In [21]:
# Cleaned occupancy schedules by land use type
for land_type in land_types:
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(111)
    df = occu_profiles_clean[land_type]
    ax.step(hr, df.mean(axis=0), color=[0, 0.45, 0.7], label='average')
    ax.fill_between(
        hr, df.min(axis=0), df.max(axis=0),
        step='pre', color=[0, 0.45, 0.7], alpha=0.5, label='range'
    )
    ax.set_xticks(range(0, 24, 2))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('hour', fontsize=14)
    ax.set_ylabel('occupancy', fontsize=14)
    ax.legend(fontsize=14)
    ax.set_xlim([0,23])
    ax.set_ylim([df.min().min() - 1,df.max().max() + 1])
    ax.set_title(land_type, fontsize=18)
    fig.savefig('.\output\occupancy_clean_{}.png'.format(land_type.lower()), bbox_inches='tight', dpi=600)
    plt.close()

In [22]:
# Plot average occupancy profiles with 95% range
for land_type in land_types:
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(111)
    df = occu_profiles_clean_norm[land_type]
    ax.step(hr, df.mean(axis=0), color=[0, 0.45, 0.7], label='average')
    ax.fill_between(
        hr, df.quantile(0.025, axis=0), df.quantile(0.975, axis=0),
        step='pre', color=[0, 0.45, 0.7], alpha=0.5, label='95% range'
    )
    ax.set_xticks(range(0, 24, 2))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('hour', fontsize=14)
    ax.set_ylabel('occupancy percentage', fontsize=14)
    ax.legend(fontsize=14)
    ax.set_xlim([0,23])
    ax.set_title(land_type, fontsize=18)
    fig.savefig('.\output\occupancy_clean_norm_{}.png'.format(land_type.lower()), bbox_inches='tight', dpi=600)
    plt.close()

In [23]:
# Export cleaned and normalized data
for land_type in land_types:
    csv_out_clean = '.\output\Detroit_occupancy_clean_{}.csv'.format(land_type.lower())
    csv_out_clean_norm = '.\output\Detroit_occupancy_clean_norm_{}.csv'.format(land_type.lower())
    occu_profiles_clean[land_type].to_csv(csv_out_clean)
    occu_profiles_clean_norm[land_type].to_csv(csv_out_clean_norm)

In [24]:
occu_profiles_clean['BUSINESS'].max(axis=1).value_counts()

4.0     107
5.0      41
6.0      20
7.0       9
10.0      3
12.0      3
11.0      2
8.0       2
14.0      1
15.0      1
13.0      1
9.0       1
dtype: int64

In [25]:
occu_profiles_clean['RESIDENTIAL-MULTI'].max(axis=1).value_counts()

4.0     133
5.0      82
6.0      35
7.0      26
8.0      11
9.0      10
10.0      8
11.0      7
15.0      3
12.0      3
13.0      2
19.0      1
28.0      1
16.0      1
14.0      1
dtype: int64

In [26]:
occu_profiles_clean['BUSINESS'].max(axis=1).value_counts().sum()

191

In [27]:
occu_profiles_clean['RESIDENTIAL-MULTI'].max(axis=1).value_counts().sum()

324

In [28]:
# Reference schedules from prototype building models
ref_sches = {
    'occupancy': {
            'BUSINESS': [
                0, 0, 0, 0, 0, 0, 0.11, 0.21, 1, 1, 1, 1,
                0.53, 1, 1, 1, 1, 0.32, 0.11, 0.11, 0.11, 0.11, 0.05, 0
            ],
            'RESIDENTIAL-MULTI': [
                1, 1, 1, 1, 1, 1, 1, 0.85, 0.39, 0.25, 0.25, 0.25,
                0.25, 0.25, 0.25, 0.25, 0.30, 0.52, 0.87, 0.87, 0.87, 1, 1, 1
            ]
    },
    'light': {
            'BUSINESS': [
                0.18, 0.18, 0.18, 0.18, 0.18, 0.23, 0.23, 0.42, 0.9, 0.9, 0.9, 0.9,
                0.8, 0.9, 0.9, 0.9, 0.9, 0.61, 0.42, 0.42, 0.32, 0.32, 0.23, 0.18
            ],
            'RESIDENTIAL-MULTI': [
                0.011, 0.011, 0.011, 0.011, 0.034, 0.074, 0.079, 0.074, 0.034, 0.023, 0.023, 0.023,
                0.023, 0.023, 0.023, 0.040, 0.079, 0.113, 0.153, 0.181, 0.181, 0.124, 0.068, 0.028
                
            ]
    },
    'plug': {
            'BUSINESS': [
                0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1, 1, 1, 1,
                0.94, 1, 1, 1, 1, 0.5, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2
            ],
            'RESIDENTIAL-MULTI': [
                0.45, 0.41, 0.39, 0.38, 0.38, 0.43, 0.54, 0.65, 0.66, 0.67, 0.69, 0.70,
                0.69, 0.66, 0.65, 0.68, 0.80, 1.00, 1.00, 0.93, 0.89, 0.85, 0.71, 0.58
            ]
    },
}

In [29]:
# Adjust lighting and plug load schedules using KNN regression
def adjust_sche(elec_ref, occu_ref, occu, n=3):
    hours_adjusted = np.asarray(range(1, 25)) / 24 * np.pi
    occu_ref_array = np.asarray(occu_ref)
    elec_ref_array = np.asarray(elec_ref)
    elec = []
    for i in range(24):
        dist = np.sqrt(
            np.sin(abs(hours_adjusted - (i+1) / 24 * np.pi)) ** 2 + (occu_ref_array - occu[i]) ** 2
        )
        try:
            elec.append(elec_ref_array[dist == 0][0])
        except IndexError:
            idxs = np.argsort(dist)[:n]
            elec.append(np.average(elec_ref_array[idxs], weights=1/dist[idxs]).round(3))
    return elec


In [30]:
# Lighting
light_profiles_clean_norm = {}
for land_type in land_types:
    light_sche = occu_profiles_clean_norm[land_type].copy()
    for i in range(occu_profiles_clean_norm[land_type].shape[0]):
        light_sche.iloc[i, :] = adjust_sche(
            ref_sches['light'][land_type],
            ref_sches['occupancy'][land_type],
            occu_profiles_clean_norm[land_type].iloc[i, :]
        )
    light_profiles_clean_norm[land_type] = light_sche

In [31]:
# Plug
plug_profiles_clean_norm = {}
for land_type in land_types:
    plug_sche = occu_profiles_clean_norm[land_type].copy()
    for i in range(occu_profiles_clean_norm[land_type].shape[0]):
        plug_sche.iloc[i, :] = adjust_sche(
            ref_sches['plug'][land_type],
            ref_sches['occupancy'][land_type],
            occu_profiles_clean_norm[land_type].iloc[i, :]
        )
    plug_profiles_clean_norm[land_type] = plug_sche

In [32]:
# Plot average occupancy profiles with 90% range and reference schedules
for land_type in land_types:
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111)
    df = occu_profiles_clean_norm[land_type]
    ax.step(hr, df.mean(axis=0), color=[0, 0.45, 0.7], label='average')
    ax.fill_between(
        hr, df.quantile(0.05, axis=0), df.quantile(0.95, axis=0),
        step='pre', color=[0, 0.45, 0.7], alpha=0.3, label='90% range'
    )
    ax.step(hr, ref_sches['occupancy'][land_type], color='k', linestyle='--', label='DOE REF')
    ax.set_xticks(range(0, 24, 2))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('hour', fontsize=14)
    ax.set_ylabel('occupancy percentage', fontsize=14)
    ax.legend(fontsize=12)
    ax.set_xlim([0,23])
    ax.set_title(land_type, fontsize=18)
    fig.savefig('.\output\occupancy_clean_norm_with_ref_{}.png'.format(land_type.lower()), bbox_inches='tight', dpi=600)
    plt.close()

In [33]:
# Plot average lighting profiles with 90% range and reference schedules
for land_type in land_types:
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111)
    df = light_profiles_clean_norm[land_type]
    ax.step(hr, df.mean(axis=0), color=[0, 0.45, 0.7], label='average')
    ax.fill_between(
        hr, df.quantile(0.05, axis=0), df.quantile(0.95, axis=0),
        step='pre', color=[0, 0.45, 0.7], alpha=0.3, label='90% range'
    )
    ax.step(hr, ref_sches['light'][land_type], color='k', linestyle='--', label='DOE REF')
    ax.set_xticks(range(0, 24, 2))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('hour', fontsize=14)
    ax.set_ylabel('lighting percentage', fontsize=14)
    ax.legend(fontsize=12)
    ax.set_xlim([0,23])
    ax.set_title(land_type, fontsize=18)
    fig.savefig('.\output\lighting_clean_norm_with_ref_{}.png'.format(land_type.lower()), bbox_inches='tight', dpi=600)
    plt.close()

In [34]:
# Plot average plug load profiles with 90% range and reference schedules
for land_type in land_types:
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111)
    df = plug_profiles_clean_norm[land_type]
    ax.step(hr, df.mean(axis=0), color=[0, 0.45, 0.7], label='average')
    ax.fill_between(
        hr, df.quantile(0.05, axis=0), df.quantile(0.95, axis=0),
        step='pre', color=[0, 0.45, 0.7], alpha=0.3, label='90% range'
    )
    ax.step(hr, ref_sches['plug'][land_type], color='k', linestyle='--', label='DOE REF')
    ax.set_xticks(range(0, 24, 2))
    ax.tick_params(labelsize=14)
    ax.set_xlabel('hour', fontsize=14)
    ax.set_ylabel('plug load percentage', fontsize=14)
    ax.legend(fontsize=12)
    ax.set_xlim([0,23])
    ax.set_title(land_type, fontsize=18)
    fig.savefig('.\output\plugload_clean_norm_with_ref_{}.png'.format(land_type.lower()), bbox_inches='tight', dpi=600)
    plt.close()

In [35]:
occu_profiles_clean_norm['BUSINESS'].mean(axis=0)

0     0.000000
1     0.000000
2     0.002443
3     0.005846
4     0.019368
5     0.076923
6     0.228767
7     0.485752
8     0.703579
9     0.755580
10    0.770876
11    0.767485
12    0.750518
13    0.663617
14    0.595604
15    0.481052
16    0.364566
17    0.286797
18    0.254291
19    0.196786
20    0.138565
21    0.087167
22    0.068171
23    0.059736
dtype: float64

In [36]:
light_profiles_clean_norm['BUSINESS'].mean(axis=0)

0     0.180000
1     0.180000
2     0.180000
3     0.180000
4     0.181393
5     0.239880
6     0.305571
7     0.552670
8     0.752675
9     0.821832
10    0.852948
11    0.854073
12    0.852702
13    0.834984
14    0.799602
15    0.715319
16    0.600775
17    0.551770
18    0.495712
19    0.426246
20    0.354634
21    0.295539
22    0.239974
23    0.185565
dtype: float64

In [37]:
plug_profiles_clean_norm['BUSINESS'].mean(axis=0)

0     0.500000
1     0.500000
2     0.500000
3     0.500000
4     0.513916
5     0.984073
6     1.000000
7     1.000000
8     0.999723
9     0.995257
10    0.993555
11    0.984450
12    0.969565
13    0.928005
14    0.856309
15    0.707984
16    0.494838
17    0.412660
18    0.333157
19    0.252644
20    0.209283
21    0.200497
22    0.200000
23    0.217738
dtype: float64

In [38]:
occu_profiles_clean_norm['RESIDENTIAL-MULTI'].mean(axis=0)

0     0.965503
1     0.964269
2     0.959948
3     0.957316
4     0.953921
5     0.916980
6     0.819804
7     0.611342
8     0.521815
9     0.475822
10    0.434599
11    0.423978
12    0.437045
13    0.465928
14    0.561065
15    0.635837
16    0.654915
17    0.689124
18    0.721131
19    0.749078
20    0.802984
21    0.843939
22    0.866332
23    0.885330
dtype: float64

In [39]:
light_profiles_clean_norm['RESIDENTIAL-MULTI'].mean(axis=0)

0     0.011769
1     0.011000
2     0.011000
3     0.012330
4     0.035182
5     0.069694
6     0.071935
7     0.055892
8     0.042981
9     0.036676
10    0.029691
11    0.026574
12    0.025941
13    0.029636
14    0.054302
15    0.097716
16    0.116015
17    0.132944
18    0.144725
19    0.164867
20    0.165590
21    0.134009
22    0.073713
23    0.031781
dtype: float64

In [40]:
plug_profiles_clean_norm['RESIDENTIAL-MULTI'].mean(axis=0)

0     0.454216
1     0.410938
2     0.390519
3     0.380571
4     0.385392
5     0.446373
6     0.566580
7     0.626991
8     0.649895
9     0.658543
10    0.675423
11    0.688157
12    0.686312
13    0.682028
14    0.744580
15    0.868210
16    0.917451
17    0.959080
18    0.958185
19    0.935019
20    0.900571
21    0.846904
22    0.714898
23    0.580907
dtype: float64